In [2]:
import pandas as pd
import cv2
import numpy as np
from keras import Sequential
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [5]:
# Load the CSV file into a pandas DataFrame
df = pd.read_csv("dorsal_right_32.csv")

In [6]:
# Create a dictionary to store the count of images for each ID
id_counts = {}

# Iterate through the DataFrame to count the number of images for each ID
for idx, row in df.iterrows():
    id_num = row["ID"]
    if id_num in id_counts:
        id_counts[id_num] += 1
    else:
        id_counts[id_num] = 1

# Create a list to store the indices of the test data
test_indices = []

# Iterate through the DataFrame to find the last 2 images for each ID
for idx, row in df.iterrows():
    id_num = row["ID"]
    if id_counts[id_num] > 2:
        id_counts[id_num] -= 1
    else:
        test_indices.append(idx)

# Split the DataFrame into training and testing datasets
train_df = df.drop(test_indices)
test_df = df.iloc[test_indices]

In [7]:
# Pre-process the images and resize them
def preprocess_image(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224,224))
    return img

In [10]:
# Prepare the train and test data
train_data = []
for i, row in train_df.iterrows():
    img_path = "/content/drive/MyDrive/Colab Notebooks/hands/" + row["ImageName"]
    img = preprocess_image(img_path)
    train_data.append(img)

train_data = np.array(train_data)
train_labels = train_df["ID"].values

test_data = []
for i, row in test_df.iterrows():
    img_path = "/content/drive/MyDrive/Colab Notebooks/hands/" + row["ImageName"]
    img = preprocess_image(img_path)
    test_data.append(img)

test_data = np.array(test_data)
test_labels = test_df["ID"].values

In [11]:
# Split the train data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.15, random_state=42)

# Load the VGG16 model with pre-trained weights
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze the layers
for layer in vgg_model.layers:
    layer.trainable = False

# Add the fully connected layers
model = Sequential()
model.add(vgg_model)
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(32, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

# Train the model
history = model.fit(train_data, train_labels, epochs=10, validation_data=(val_data, val_labels))

model.save("hand_identification_model.h5")


58889256/58889256 [==============================] - 3s 0us/step


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/10
5/5 [==============================] - 16s 1s/step - loss: 15.8216 - accuracy: 0.2975 - val_loss: 5.2925 - val_accuracy: 0.5862
Epoch 2/10
5/5 [==============================] - 1s 183ms/step - loss: 1.0975 - accuracy: 0.8861 - val_loss: 3.6665 - val_accuracy: 0.7241
Epoch 3/10
5/5 [==============================] - 1s 180ms/step - loss: 0.1015 - accuracy: 0.9810 - val_loss: 2.7217 - val_accuracy: 0.7241
Epoch 4/10
5/5 [==============================] - 1s 180ms/step - loss: 0.0459 - accuracy: 0.9873 - val_loss: 1.9361 - val_accuracy: 0.7931
Epoch 5/10
5/5 [==============================] - 1s 182ms/step - loss: 3.6830e-05 - accuracy: 1.0000 - val_loss: 1.6608 - val_accuracy: 0.8276
Epoch 6/10
5/5 [==============================] - 1s 181ms/step - loss: 2.5532e-05 - accuracy: 1.0000 - val_loss: 1.5780 - val_accuracy: 0.8621
Epoch 7/10
5/5 [==============================] - 1s 181ms/step - loss: 1.2319e-05 - accuracy: 1.0000 - val_loss: 1.5434 - val_accuracy: 0.8621
Epoch 8/1

In [12]:
# Evaluate the model on test data
test_loss, test_acc = model.evaluate(test_data, test_labels)
print('Test Accuracy: ', test_acc)
print('Test loss:', test_loss)

2/2 [==============================] - 0s 153ms/step - loss: 4.8266 - accuracy: 0.7969
Test Accuracy:  0.796875
Test loss: 4.826579570770264
